In [2]:
pip install pandas


In [3]:
pip install keras


  Using cached keras-3.10.0-py3-none-any.whl.metadata (6.0 kB)
  Using cached absl_py-2.3.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached namex-0.1.0-py3-none-any.whl.metadata (322 bytes)
Using cached keras-3.10.0-py3-none-any.whl (1.4 MB)
Using cached absl_py-2.3.0-py3-none-any.whl (135 kB)
Using cached namex-0.1.0-py3-none-any.whl (5.9 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install tensorflow


  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/376.0 MB ? eta -:--:--
    --------------------------------------- 5.0/376.0 MB 27.4 MB/s eta 0:00:14
   -- ------------------------------------- 22.8/376.0 MB 60.1 MB/s eta 0:00:06
   ---- ----------------------------------- 38.3/376.0 MB 65.8 MB/s eta 0:00:06
   ----- ---------------------------------- 50.3/37

In [3]:
import pandas as pd

In [4]:
datos = pd.read_csv("simulacionEconomica.csv")
datos.head()


,demanda_preajuste,inventario_preajuste,produccion_preajuste,precio_preajuste,volatilidad_precio,subida_brusca_precio,escasez
0,1381.0,0.0,1274.0,10.0,0.50000,0,0
1,1379.0,0.0,1425.0,11.0,0.47140,0,0
2,1377.0,33.0,1543.0,11.0,0.50000,0,0
3,1390.0,141.0,1555.0,10.0,0.74833,0,0
4,1362.0,214.0,1507.0,9.0,1.06719,0,0


In [5]:
datos.shape

(35000, 7)

In [6]:
atributos = datos.loc[:, 'demanda_preajuste':'volatilidad_precio']
atributos.head()

,demanda_preajuste,inventario_preajuste,produccion_preajuste,precio_preajuste,volatilidad_precio
0,1381.0,0.0,1274.0,10.0,0.50000
1,1379.0,0.0,1425.0,11.0,0.47140
2,1377.0,33.0,1543.0,11.0,0.50000
3,1390.0,141.0,1555.0,10.0,0.74833
4,1362.0,214.0,1507.0,9.0,1.06719


In [ ]:
objetivo = datos.loc[:, 'subida_brusca_precio':'escasez']

objetivo.head()

,subida_brusca_precio,escasez
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
    atributos, objetivo,
    test_size=.2
)

In [10]:
atributos_entrenamiento.head()

,demanda_preajuste,inventario_preajuste,produccion_preajuste,precio_preajuste,volatilidad_precio
33227,1505.0,68.0,1597.0,12.0,1.52561
28835,1342.0,8.0,1371.0,42.0,4.83839
15809,1412.0,4.0,1492.0,15.0,1.95078
30649,2213.0,53.0,2230.0,12.0,0.97980
31668,2311.0,0.0,2299.0,36.0,4.98874


In [11]:
atributos_entrenamiento.shape

(28000, 5)

In [12]:
from keras import Sequential, Input, Model
from keras.layers import Dense, Dropout

In [13]:
from keras.layers import Normalization
import numpy as np

In [14]:
normalizador = Normalization()
normalizador.adapt(atributos_entrenamiento.to_numpy())

In [15]:
np.mean(normalizador(atributos_entrenamiento), axis=0)

array([-3.6903788e-08,  7.4377816e-08,  4.7428266e-08,  1.2126884e-08,
        4.4809920e-09], dtype=float32)

In [16]:
np.var(normalizador(atributos_entrenamiento), axis=0)

array([0.9999887 , 1.000028  , 0.99999315, 0.9999985 , 1.0000254 ],
      dtype=float32)

In [18]:
red_mercado = Sequential()
red_mercado.add(Input(shape=(5,)))            # Entrada: 5 variables
red_mercado.add(normalizador)
red_mercado.add(Dense(64, activation='relu')) # Capa oculta 1 con 64 neuronas
red_mercado.add(Dropout(0.2))                 # Regularización
red_mercado.add(Dense(32, activation='relu')) # Capa oculta 2 con 32 neuronas
red_mercado.add(Dense(2, activation='sigmoid')) 

In [19]:
red_mercado.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization (Normalization)   │ (None, 5)              │            11 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,541 (9.93 KB)

 Trainable params: 2,530 (9.88 KB)

 Non-trainable params: 11 (48.00 B)

In [25]:
from keras.metrics import Recall

In [26]:
red_mercado.compile(optimizer='SGD', loss='binary_crossentropy', 
                    metrics=['binary_accuracy', Recall()])

In [27]:
red_mercado.fit(atributos_entrenamiento, objetivo_entrenamiento,
                batch_size=128, epochs=20, validation_split=0.2)

Epoch 1/20


175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - binary_accuracy: 0.8783 - loss: 0.2501 - recall: 0.7023 - val_binary_accuracy: 0.8815 - val_loss: 0.2382 - val_recall: 0.7117
Epoch 2/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8815 - loss: 0.2460 - recall: 0.7144 - val_binary_accuracy: 0.8821 - val_loss: 0.2370 - val_recall: 0.7120
Epoch 3/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8820 - loss: 0.2444 - recall: 0.7131 - val_binary_accuracy: 0.8830 - val_loss: 0.2359 - val_recall: 0.7142
Epoch 4/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8832 - loss: 0.2456 - recall: 0.7099 - val_binary_accuracy: 0.8838 - val_loss: 0.2350 - val_recall: 0.7148
Epoch 5/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8807 - loss: 0.2456 - recall: 0.7121 - val_binary_accuracy: 0.8844 - val_loss: 0.2339 - val_recall: 0.7164
Epoch 6/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8805 - loss: 0.2444 - recall: 0.7021 -

In [47]:
red_mercado.evaluate(atributos_prueba, objetivo_prueba)

219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - binary_accuracy: 0.8959 - loss: 0.2181 - recall: 0.7384


[0.2223670482635498, 0.8926428556442261, 0.7230769395828247]

In [59]:
predicciones = red_mercado.predict(datos.iloc[1510:1520, 0:5])
predicciones

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


array([[6.41664028e-01, 4.24076691e-02],
       [4.15156752e-01, 4.64736577e-03],
       [4.30262744e-01, 4.58067283e-03],
       [3.13498497e-01, 1.60448020e-03],
       [4.74691659e-01, 5.53128915e-03],
       [7.18821287e-01, 4.90281358e-02],
       [5.92682421e-01, 1.37078315e-02],
       [1.41106471e-01, 1.04796200e-04],
       [2.53888637e-01, 3.15749843e-04],
       [7.71934986e-01, 6.76140115e-02]], dtype=float32)

In [60]:
predicciones_binarias = (predicciones >= 0.5).astype(int)

In [61]:
predicciones_binarias

array([[1, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [1, 0]])